In [2]:
import pandas as pd
#from google.cloud import bigquery
#from google.oauth2 import service_account
#from params.py import PROJECT_ID, TABLE_ID
import sys; sys.path.append('..')
#sys.path.append('../..')
#from params import PROJECT_ID, TABLE_ID
#from data_extraction import get_coordinates_from_collection
#from utils import delete_local_enchantillon


In [36]:
import pickle

def upload_model(model, name_of_model):
    with open(f'model.{name_of_model}', 'wb') as file:
        pickle.dump(model, file)


def load_model(name_of_the_model=False):
    if name_of_the_model == False:
        with open('model.randomfo', 'rb') as file:
            loaded_model = pickle.load(file)
        return loaded_model
    else:
        with open(name_of_the_model, 'rb') as file:
            loaded_model = pickle.load(file)
        return loaded_model

In [37]:
model_eval = load_model('model.model_LSTM9')

In [43]:
import os
import cv2
import string
import uuid
import time
import numpy as np
import mediapipe as mp
import pandas as pd

In [44]:
def get_coordinates_live(frame):
    """
    Récupère toutes les coords de la main d'une image et stocke dans un dictionnaire
    """
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)

    #img = cv2.imread(image)

    #img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = hands.process(img_rgb)

    coords={}
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            wrist_x = hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].x
            wrist_y = hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].y
            wrist_z = hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].z

            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y
                z = hand_landmarks.landmark[i].z

                coords[f"x_{i}"]= x - wrist_x
                coords[f"y_{i}"]= y - wrist_y
                coords[f"z_{i}"]= z - wrist_z

    return coords

In [61]:
def gmas_live(list_signs):
    
    cap = cv2.VideoCapture(0)

    ok = 'o'
    while True:
        ret, frame = cap.read()
        cv2.putText(frame, f'Ready? Press "{ok}" ! :)', (100, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 255, 0), 3,
                    cv2.LINE_AA)
        cv2.imshow('frame', frame)
        if cv2.waitKey(25) == ord(ok):
            break
    
    
    please_stop = 'q'
    coord_frame = []
    dataset_size = 10
    counter = 0
    while counter < dataset_size:
        coord_frame = []
        ret, frame = cap.read()
        
        cv2.imshow('frame', frame)
        coords = get_coordinates_live(frame)
        coord_frame.append(coords)
        df = pd.DataFrame(coord_frame)
        
        res = model_eval.predict(df)
        guess = list_signs[np.argmax(res)]
        
        cv2.putText(frame, f'My guess "{guess}" ! :)', (100, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 255, 0), 3,
                    cv2.LINE_AA)
        
        if cv2.waitKey(2000) == ord(please_stop):
            break
        
        print(f' image : {counter}, la lettre est {guess}', )
        counter += 1

    cv2.destroyAllWindows()
    cap.release()
    cv2.destroyAllWindows()
    

pass



In [62]:
list_signs = ["A", "B", "C"]


In [63]:
gmas_live(list_signs)
cap = cv2.VideoCapture(0)
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 56ms/step
 image : 0, la lettre est A
1/1 [==============================] - 0s 63ms/step
 image : 1, la lettre est A
1/1 [==============================] - 0s 63ms/step
 image : 2, la lettre est C
1/1 [==============================] - 0s 65ms/step
 image : 3, la lettre est B
1/1 [==============================] - 0s 55ms/step
 image : 4, la lettre est C
1/1 [==============================] - 0s 51ms/step
 image : 5, la lettre est C
1/1 [==============================] - 0s 56ms/step
 image : 6, la lettre est C
1/1 [==============================] - 0s 54ms/step
 image : 7, la lettre est A
1/1 [==============================] - 0s 52ms/step
 image : 8, la lettre est A
1/1 [==============================] - 0s 53ms/step
 image : 9, la lettre est B


: 

In [5]:
#cv2.imshow('frame', frame)
#cap.release()
cv2.destroyAllWindows()